In [1]:
import numpy as np
import pandas as pd
from pyhanlp import *
import re

In [2]:
#朴素贝叶斯
NaiveBayesClassifier = SafeJClass('com.hankcs.hanlp.classification.classifiers.NaiveBayesClassifier')

In [3]:
training_data_path = './data/training_data/ChnSentiCorp'
# training_data_path = './data/training_data/blog'
# test_data_paths = ['./data/test_data/0926.xlsx','./data/test_data/1017.xlsx','./data/test_data/1107.xlsx','./data/test_data/1114.xlsx','./data/test_data/1128.xlsx','./data/test_data/1205.xlsx','./data/test_data/1212.xlsx']

test_data_paths = ['./data/test_data/0926.xlsx','./data/test_data/1017.xlsx','./data/test_data/1107.xlsx','./data/test_data/1114.xlsx']

In [4]:
def train_classifier(path):
    classifier = NaiveBayesClassifier()
    classifier.train(training_data_path)
    model = classifier.getModel()
    return NaiveBayesClassifier(model)

In [5]:
def read_and_adjust_sheet(test_data_path):
    # 只读取回帖列
    sheet = pd.read_excel(test_data_path, usecols=[2], header = None)
    # print(sheet)
    sheet = sheet.drop([0])
    sheet = sheet.rename(columns={0: 'replies'})
    
    # 清理数据
    clean_reply = []
    for index,row in sheet.iterrows():
        reply = row.str.cat(sep=',')
        # reply = row[0]
        # print(row[0])
        # 自动回复内容删除  
        auto = '(\s)*回复\s[0-9]*(\s)*#(\s)*的帖子(\n)*'
        reply = re.sub(auto,'',reply)
        clean_reply.append(reply)
    
    clean_replies = np.array(clean_reply)
    sheet.insert(0, 'clean_replies',clean_replies)
    
    # 删除字数小于15字的帖子
    for index,row in sheet.iterrows():
        clean_data = row['clean_replies']
        if(len(clean_data)<15):
            sheet = sheet.drop(index)
        
    return sheet

In [6]:
def predict_sentiment(classifier, sheet):
    sentiment = []
    for index,row in sheet.iterrows():
        sentiment.append(classifier.classify(row.str.cat(sep=',')))
    return sentiment

In [7]:
def extract_keywords(sheet):
    keywords = []
    for index,row in sheet.iterrows():
        #提取2个关键词
        extraction = list(HanLP.extractKeyword(row[0], 2))
        temp = ','.join(extraction)
        keywords.append(temp)
        #提取1个关键词
        #keywords.append(HanLP.extractKeyword(row.str.cat(sep=','), 1))
    return keywords

In [8]:
def write(keywords,sentiment,test_data_path):
    # 在表格中加入新列(结果列)
    sentiment_results = np.array(sentiment)
    keywords_results = np.array(keywords)
    sheet.insert(1, 'sentiment', sentiment_results)
    sheet.insert(1, 'keywords',keywords_results)
    
    # 设置写回路径
    root_path = './data/results_v1/'
    suffix = test_data_path[-9:-5]+'_output_v1.xlsx'
    back_path = root_path+suffix
    #print(back_path)
    sheet.to_excel(back_path) 

In [9]:
if __name__ == '__main__':
    classifier = train_classifier(training_data_path)
    
    for test_data_path in test_data_paths:
        sheet = read_and_adjust_sheet(test_data_path)
        sentiment = predict_sentiment(classifier,sheet)
        keyword = extract_keywords(sheet)
        write(keyword,sentiment,test_data_path)


In [10]:
sheet

,clean_replies,keywords,sentiment,replies
1,假如是学习单词，为何不选择百词斩呢。这个或许在记单词方面更具普遍性，被试对象的选择更加方便些,"选择,单词",负面,假如是学习单词，为何不选择百词斩呢。这个或许在记单词方面更具普遍性，被试对象的选择更加方便些
4,实验类型是不是还有前后测设计，以保证初始能力水平大体一致,"设计,保证",负面,实验类型是不是还有前后测设计，以保证初始能力水平大体一致
5,调查过若干考研的人，百词斩的图片记忆只适合短期记忆，如果要形成长期记忆，那么扇贝的效果比百词...,"词,斩",负面,回复 2 # 的帖子\n调查过若干考研的人，百词斩的图片记忆只适合短期记忆，如果要形成长期记...
6,有的，写的时候没有写清楚 ，但是我们在实验模式与研究流程中是有指明的,"模式,实验",负面,回复 5 # 的帖子\n有的，写的时候没有写清楚 ，但是我们在实验模式与研究流程中是有指明的
7,只选取20名研究生是不是有点少？如果可以，建议多选点被试对象,"少,有点",负面,只选取20名研究生是不是有点少？如果可以，建议多选点被试对象
8,实验方案做的还是很好的。但是，实验周期定为5天好像有点少了，还有就是单词的应该选择实验对象都...,"实验,单词",负面,实验方案做的还是很好的。但是，实验周期定为5天好像有点少了，还有就是单词的应该选择实验对象都...
9,实验思路清晰，也很详尽，值得我们学习。,"值得,清晰",正面,回复 4 # 的帖子\n实验思路清晰，也很详尽，值得我们学习。
10,实验思路清晰，过程也较为详细。 \n研究问题不就一个问题吗？为什么标记“1”呢？单词水平相同...,"水平,问题",负面,实验思路清晰，过程也较为详细。 \n研究问题不就一个问题吗？为什么标记“1”呢？单词水平相同...
11,学习时间的长短也是无关变量吧！而且在设计实验时不能设置为都有传统授课，而实验组还有App辅助...,"学习,传统",负面,回复 1 # 的帖子\n学习时间的长短也是无关变量吧！而且在设计实验时不能设置为都有传统授课...
12,百词斩和扇贝单词都是英语学习工具，选择扇贝是我们使用过这个软件，也觉得适用于自主学习研究。,"学习,扇贝",负面,回复 2 # 的帖子\n百词斩和扇贝单词都是英语学习工具，选择扇贝是我们使用过这个软件，也觉...
